In [2]:
import pandas as pd
import numpy as np
import re
import string

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.pipeline import make_pipeline

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

from bs4 import BeautifulSoup
from langdetect import detect
from urllib.parse import urlsplit

from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Pre Processing

In [3]:
df = pd.read_csv("../Dataset/IMDB Dataset.csv")
df.head()
df = df.rename(columns={'review': 'OriginalReviews'})
df = df.rename(columns={'sentiment': 'OutputSentiment'})
df_subset = df.sample(n=5000).reset_index(drop=True)
df_subset.head()
df_subset['OutputSentiment'].value_counts()

OutputSentiment
positive    2532
negative    2468
Name: count, dtype: int64

In [4]:
#lowercase
df_subset["OriginalReviews"]=df_subset["OriginalReviews"].apply(lambda x:x.lower())

#Remove punctuations
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].str.replace('[{}]'.format(string.punctuation), ' ')

# Remove numbers from the 'OriginalReviewss' column
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].str.replace('\d+', '')

In [5]:
# Function to remove stopwords from a text
def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)

# Apply the remove_stopwords function to the 'OriginalReviews' column
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(remove_stopwords)

In [6]:
def remove_urls(text):
    # Define a regular expression pattern to match URLs
    url_pattern = re.compile(r'https?://\S+|www\.\S+')

    # Find all matches in the text
    urls = re.findall(url_pattern, text)

    # Remove URLs from the text
    text_without_urls = re.sub(url_pattern, '', text)

    return text_without_urls

# Example usage
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(remove_urls)

In [7]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

df_subset["OriginalReviews"] = df_subset["OriginalReviews"].apply(remove_html_tags)

C:\Users\sdidd\AppData\Local\Temp\ipykernel_7148\3731208440.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


In [8]:
def clean_text(text):
    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return text

df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(clean_text)

In [9]:
def remove_extra_whitespaces(text):
    # Use regular expression to replace multiple whitespaces with a single space
    return re.sub(r'\s+', ' ', text).strip()

df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(remove_extra_whitespaces)

In [10]:
def filter_non_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

# Create a boolean mask for non-English OriginalReviewss
mask = df_subset['OriginalReviews'].apply(filter_non_english)

# Create a new DataFrame containing only English OriginalReviewss
df_subset = df_subset[mask]

In [11]:
# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to get the part of speech for WordNet lemmatizer
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun if the part of speech is not found

# Function to lemmatize a text
def lemmatize_text(text):
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    lemmatized_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in pos_tags]
    return ' '.join(lemmatized_tokens)

# Apply lemmatization to the 'text' column
df_subset['OriginalReviews'] = df_subset['OriginalReviews'].apply(lemmatize_text)

In [12]:
df_subset

,OriginalReviews,OutputSentiment
0,yes m go 10 s last year watch quite comedy lea...,positive
1,possibly one bad giant killer animal movie eve...,negative
2,inspire director movie excite watch criticism ...,negative
3,arguably bad major ava gardner film yes gorgeo...,negative
4,guy whoever even think watch movie already get...,negative
...,...,...
4995,decide watch ultralow budget film poverty row ...,negative
4996,suggestion movie really intended comedy nt pro...,negative
4997,disappointment br br film seem try copy cut ed...,negative
4998,first film accord slick say best film worldfes...,negative


In [13]:
df_subset.to_csv("../csv/Preprocessed_data.csv",index=False)

## Feature Extraction Using TF-IDF

In [14]:
preprocessed = pd.read_csv('../csv/Preprocessed_data.csv')

In [15]:
num_features_to_keep = 13000

# Create a pipeline with TfidfVectorizer and SelectKBest
pipeline = make_pipeline(TfidfVectorizer(), SelectKBest(f_classif, k=num_features_to_keep))

# Fit and transform your data
X_transformed = pipeline.fit_transform(preprocessed['OriginalReviews'], preprocessed['OutputSentiment'])

# Get the selected feature names
selected_feature_names = pipeline.named_steps['tfidfvectorizer'].get_feature_names_out()[pipeline.named_steps['selectkbest'].get_support()]

# Create a DataFrame with the selected features
selected_features_df = pd.DataFrame(X_transformed.toarray(), columns=selected_feature_names)

# Concatenate the existing DataFrame with the new selected features DataFrame
tfidf_df_13k = pd.concat([preprocessed, selected_features_df], axis=1)

tfidf_df_13k.head()

tfidf_df_13k.to_csv("../csv/tfidf_df_13k.csv")

## CONNOTATIONS

In [16]:
# Download the VADER lexicon (run this once)
nltk.download('vader_lexicon')
delimiter = '\t'

# Read the text file into a DataFrame
positive = pd.read_csv(r'..\Connotations\positive-words.txt', sep=delimiter, names=['words'])
negative = pd.read_csv(r'..\Connotations\negative-words.txt', sep=delimiter, names=['words'])
connotations = pd.read_csv(r"..\Connotations\connotations.csv")

word_emotion_map = dict(zip(connotations['word'], connotations['emotion']))

def update_counts(review):
    positive_count = sum(1 for word in review.split() if word in word_emotion_map and word_emotion_map[word] == 'positive')
    negative_count = sum(1 for word in review.split() if word in word_emotion_map and word_emotion_map[word] == 'negative')
    return positive_count, negative_count

tfidf_df_13k[['Positive_Connotation_Count', 'Negative_Connotation_Count']] = tfidf_df_13k['OriginalReviews'].apply(update_counts).tolist()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\sdidd\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [17]:
# Load positive and negative words from files
positive_words_df = pd.read_csv(r'..\Connotations\positive-words.txt', header=None, names=['words'])
negative_words_df = pd.read_csv(r'..\Connotations\negative-words.txt', header=None, names=['words'])

# Convert DataFrame columns to sets
positive_words = set(positive_words_df['words'].tolist())
negative_words = set(negative_words_df['words'].tolist())

# Assuming 'tfidf_df_13k' is your DataFrame
# Define a function to update counts based on positive and negative words
def update_word_counts(review):
    positive_count = sum(1 for word in review.split() if word in positive_words)
    negative_count = sum(1 for word in review.split() if word in negative_words)
    return positive_count, negative_count

# Apply the function to the 'OriginalReviews' column and unpack the result into two new columns
tfidf_df_13k[['Positive_Word_Count', 'Negative_Word_Count']] = tfidf_df_13k['OriginalReviews'].apply(update_word_counts).tolist()

In [18]:
from nltk.sentiment import SentimentIntensityAnalyzer
# Use VADER for sentiment analysis
sid = SentimentIntensityAnalyzer()

def vader_sentiment(review):
    scores = sid.polarity_scores(review)
    return scores['pos'] *100, scores['neg'] * 100

# Apply the function to the 'OriginalReviews' column and unpack the result into two new columns
tfidf_df_13k[['Positive_VADER_Count', 'Negative_VADER_Count']] = tfidf_df_13k['OriginalReviews'].apply(vader_sentiment).tolist()

tfidf_df_13k.to_csv("../csv/tfidf_df_13k_connotations_vader.csv")

In [19]:
text = tfidf_df_13k.iloc[4993]['OriginalReviews']

In [20]:
tfidf_df_13k = pd.read_csv("../csv/tfidf_df_13k.csv")

In [21]:
tfidf_df_13k_connotations = pd.read_csv('../csv/tfidf_df_13k_connotations_vader.csv')

In [22]:
tfidf_df_13k_connotations = tfidf_df_13k_connotations.drop('Unnamed: 0',axis=1)

In [23]:
tfidf_df_13k_connotations

,OriginalReviews,OutputSentiment,007,010,07,10,100,1000,100000,1000000,...,zulu,zwick,zz,zzzzzzzzzzzz,Positive_Connotation_Count,Negative_Connotation_Count,Positive_Word_Count,Negative_Word_Count,Positive_VADER_Count,Negative_VADER_Count
0,yes m go 10 s last year watch quite comedy lea...,positive,0.0,0.0,0.0,0.099588,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,36,20,11,9,37.7,15.4
1,possibly one bad giant killer animal movie eve...,negative,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25,24,4,10,19.7,20.1
2,inspire director movie excite watch criticism ...,negative,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,17,14,7,3,28.9,16.5
3,arguably bad major ava gardner film yes gorgeo...,negative,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,26,28,6,7,24.3,20.4
4,guy whoever even think watch movie already get...,negative,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,18,24,2,8,13.7,11.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,decide watch ultralow budget film poverty row ...,negative,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,71,56,13,22,22.5,12.8
4996,suggestion movie really intended comedy nt pro...,negative,0.0,0.0,0.0,0.098086,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25,16,3,3,20.5,10.8
4997,disappointment br br film seem try copy cut ed...,negative,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,14,19,0,7,13.4,21.0
4998,first film accord slick say best film worldfes...,negative,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,46,41,13,13,21.9,22.2


In [24]:
df_statistical = tfidf_df_13k_connotations.drop(columns=['OriginalReviews','Positive_Connotation_Count','Negative_Connotation_Count','Positive_Word_Count','Negative_Word_Count','Positive_VADER_Count','Negative_VADER_Count'], axis=1)
df_statistical.head()

,OutputSentiment,007,010,07,10,100,1000,100000,1000000,1010,...,zombified,zone,zoneish,zoom,zu,zukhov,zulu,zwick,zz,zzzzzzzzzzzz
0,positive,0.0,0.0,0.0,0.099588,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,negative,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,negative,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,negative,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,negative,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
label = LabelEncoder()
df_statistical['OutputSentiment'] = label.fit_transform(df_statistical['OutputSentiment'])

## CHI SQAURE

In [26]:
# This will get the top 5000 relavant features out of the sample
chi2_selector = SelectKBest(chi2, k=5000)

# This will transform the dataset i.e, it will reduce the dimensions by just considering the relavant features only
X = df_statistical.drop(columns=['OutputSentiment'])
y = df_statistical['OutputSentiment']
X_5000 = chi2_selector.fit_transform(X, y)

# Get the indices of the selected features
selected_feature_indices = chi2_selector.get_support(indices=True)

# Get the names of the selected features
selected_feature_names = X.columns[selected_feature_indices]

chisq_5k = X[selected_feature_names]
chisq_5k.head()

chisq_5k = pd.concat([chisq_5k,tfidf_df_13k_connotations.iloc[:, -6:]],axis=1)
chisq_5k.head()

,007,010,07,100000,1010,10yr,110,1110,117,11th,...,zu,zulu,zwick,zzzzzzzzzzzz,Positive_Connotation_Count,Negative_Connotation_Count,Positive_Word_Count,Negative_Word_Count,Positive_VADER_Count,Negative_VADER_Count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,36,20,11,9,37.7,15.4
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25,24,4,10,19.7,20.1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,17,14,7,3,28.9,16.5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,26,28,6,7,24.3,20.4
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,18,24,2,8,13.7,11.7


In [27]:
# This will get the top 5000 relavant features out of the sample
chi2_selector = SelectKBest(chi2, k=8000)

# This will transform the dataset i.e, it will reduce the dimensions by just considering the relavant features only
X = df_statistical.drop(columns=['OutputSentiment'])
y = df_statistical['OutputSentiment']
X_8000 = chi2_selector.fit_transform(X, y)

# Get the indices of the selected features
selected_feature_indices = chi2_selector.get_support(indices=True)

# Get the names of the selected features
selected_feature_names = X.columns[selected_feature_indices]

chisq_8k = X[selected_feature_names]
chisq_8k.head()

chisq_8k = pd.concat([chisq_8k,tfidf_df_13k_connotations.iloc[:, -6:]],axis=1)
chisq_8k.head()

,007,010,07,1000,100000,1000000,1010,105,10year,10yr,...,zu,zulu,zwick,zzzzzzzzzzzz,Positive_Connotation_Count,Negative_Connotation_Count,Positive_Word_Count,Negative_Word_Count,Positive_VADER_Count,Negative_VADER_Count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,36,20,11,9,37.7,15.4
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25,24,4,10,19.7,20.1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,17,14,7,3,28.9,16.5
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,26,28,6,7,24.3,20.4
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,18,24,2,8,13.7,11.7


In [28]:
chisq_8k
columns_to_drop = ['Positive_Connotation_Count', 'Negative_Connotation_Count', 
                   'Positive_Word_Count', 'Negative_Word_Count', 
                   'Positive_VADER_Count', 'Negative_VADER_Count']

chisq_8k.drop(columns=columns_to_drop, inplace=True)

## CHI SQAURE CLASSIFICATION

In [29]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

# Multinomial Naive Bayes Classifier
nb_classifier = MultinomialNB()
nb_scores = cross_val_score(nb_classifier, chisq_8k, y, cv=5)

print("Multinomial Naive Bayes Cross-Validation Scores:")
print(nb_scores)
print("Mean Accuracy:", np.mean(nb_scores))

# k-Nearest Neighbors Classifier
knn_classifier = KNeighborsClassifier()
knn_scores = cross_val_score(knn_classifier, chisq_5k, y, cv=5)

print("\nk-Nearest Neighbors Cross-Validation Scores:")
print(knn_scores)
print("Mean Accuracy:", np.mean(knn_scores))

Multinomial Naive Bayes Cross-Validation Scores:
[0.909 0.907 0.907 0.894 0.906]
Mean Accuracy: 0.9046

k-Nearest Neighbors Cross-Validation Scores:
[0.699 0.707 0.689 0.691 0.703]
Mean Accuracy: 0.6977999999999999


In [30]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import cross_val_score, KFold
# from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression

# # Load your data
# # Assuming X and y are your features and target variables

# # Initialize models
# svm_model = SVC(kernel='linear')  # Linear SVM
# logistic_model = LogisticRegression()

# # Initialize KFold cross-validation
# kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# # Perform 5-fold cross-validation for SVM
# svm_scores = cross_val_score(svm_model, chisq_5k, y, cv=kfold)

# # Perform 5-fold cross-validation for Logistic Regression
# logistic_scores = cross_val_score(logistic_model, chisq_8k, y, cv=kfold)

# # Display the cross-validation scores
# print("SVM Cross-validation scores:", svm_scores)
# print("Logistic Regression Cross-validation scores:", logistic_scores)

# # Optionally, you can calculate mean and standard deviation of the scores
# print("SVM Mean Accuracy:", np.mean(svm_scores))
# print("SVM Standard Deviation of Accuracy:", np.std(svm_scores))
# print("Logistic Regression Mean Accuracy:", np.mean(logistic_scores))
# print("Logistic Regression Standard Deviation of Accuracy:", np.std(logistic_scores))

In [31]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import StratifiedKFold

# Assuming chisq_8k has features and y is the output

# Encode categorical labels if needed
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(chisq_8k, y_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build a simple neural network model
model = Sequential()
model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, verbose=1)

# Evaluate the model on the test set
y_pred = (model.predict(X_test_scaled) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, y_pred)

print("Neural Network Accuracy on Test Set:", accuracy)




Epoch 1/10


125/125 [==============================] - 2s 11ms/step - loss: 0.3513 - accuracy: 0.8372
Epoch 2/10
125/125 [==============================] - 1s 12ms/step - loss: 0.0177 - accuracy: 0.9952
Epoch 3/10
125/125 [==============================] - 1s 11ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 4/10
125/125 [==============================] - 2s 12ms/step - loss: 8.8024e-04 - accuracy: 1.0000
Epoch 5/10
125/125 [==============================] - 1s 11ms/step - loss: 4.8438e-04 - accuracy: 1.0000
Epoch 6/10
125/125 [==============================] - 1s 12ms/step - loss: 3.0205e-04 - accuracy: 1.0000
Epoch 7/10
125/125 [==============================] - 1s 11ms/step - loss: 2.0994e-04 - accuracy: 1.0000
Epoch 8/10
125/125 [==============================] - 1s 12ms/step - loss: 1.5611e-04 - accuracy: 1.0000
Epoch 9/10
125/125 [==============================] - 1s 12ms/step - loss: 1.2032e-04 - accuracy: 1.0000
Epoch 10/10
32/32 [==============================] - 0s 2ms/st

In [32]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

# Assuming chisq_8k has features and y is the output

# Encode categorical labels if needed
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(chisq_8k, y_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build a simple neural network model
model = Sequential()
model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(
    X_train_scaled, y_train, 
    epochs=50, batch_size=32, 
    validation_split=0.1,  # Using a portion of training set for validation
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate the model on the test set
y_pred = (model.predict(X_test_scaled) > 0.5).astype("int32")
accuracy = accuracy_score(y_test, y_pred)

print("Neural Network Accuracy on Test Set:", accuracy)


Epoch 1/50
113/113 [==============================] - 3s 16ms/step - loss: 0.3322 - accuracy: 0.8469 - val_loss: 0.1783 - val_accuracy: 0.9225
Epoch 2/50
113/113 [==============================] - 1s 12ms/step - loss: 0.0142 - accuracy: 0.9953 - val_loss: 0.1828 - val_accuracy: 0.9275
Epoch 3/50
113/113 [==============================] - 1s 12ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 0.1880 - val_accuracy: 0.9250
Epoch 4/50
32/32 [==============================] - 0s 2ms/step
Neural Network Accuracy on Test Set: 0.926
